In [ ]:
import os,sys,inspect
import random
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil
from functools import partial
import json

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест команд автоматизации для калибровок"
display(Markdown('### 💠 {}'.format(testname)))



In [ ]:
from connect_device_v2.find_device import FindDevice,parse_config
box = FindDevice()
display(box)

In [ ]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")

device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}
#inst2 = ng.instance()
#путь к файлу с сегментами
currentDir=%pwd
#resultPath= currentDir +'\\frequency_plan.seg'
portCount = int(inst.query("SERV:PORT:COUN?"))
print(device_info)
print(portCount)

In [ ]:
def check_error():
    responce = inst.query('SYST:ERR?')    
    if "No error" not in responce:        
        print(responce)
        #pass
    else:        
        pass
        #print('succes')

In [ ]:
def setup_mesuare_parametrs(con):
    con.write(f"SENS:CORR:COLL:CKIT 3")    
    check_error()
    for i in range(1,3): 
        #print("SENS:CORR:COLL:CKIT:ORD:SEL 2")
        #con.write("SENS:CORR:COLL:CKIT:ORD:SEL 2")
        #check_error()
        #print(f"SENS:CORR:COLL:CKIT:STAN{i}:DEL 0")
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:DEL 0")
        check_error()
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:LOSS 0")
        check_error()
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C0 0")
        check_error()
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C1 0")
        check_error()
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C2 0")
        check_error()
        con.write(f"SENS:CORR:COLL:CKIT:STAN{i}:C3 0")
        check_error()

In [ ]:
setup_mesuare_parametrs(inst)

In [ ]:
def kalibration_ro_for_port_list(con,ports_list:list):
    #print('port list=',ports_list)
    #print('port list=',type(ports_list))
    #print(f'SYST:CORR 0')
    con.write('SYST:CORR 0') 
    check_error()
    
#     con.write("SENS:CORR:COLL:CKIT 2")
#     check_error()
#     con.write("SENS:CORR:COLL:CKIT:ORD:SEL 1")
#     check_error()
  
    con.write(f"SENS:CORR:COLL:CKIT 3")
    check_error()
    con.write("DISP:WIND:TRAC:Y:PDIV 0.05")
    check_error()
    con.write("SENS:CORR:STAT ON")
    check_error()
    # time.sleep(2)
    #time.sleep(3) 
    for port in ports_list:     
        #print(f"SENS1:CORR:COLL:OPEN {port}")
        con.write(f"SENS1:CORR:COLL:OPEN {port}")
        check_error()
        #print(f"SENS1:CORR:COLL:METH:OPEN {port}")
        con.write(f"SENS1:CORR:COLL:METH:OPEN {port}") 
        check_error()
        #print("SENS:CORR:COLL:SAVE")
        
        con.write("SENS:CORR:COLL:SAVE")         
        check_error()
        con.write(f"CALC:PAR1:DEF s{port}{port}") 
        #print("SENS:CORR:COLL:SAVE")
        #print(con.query(f'SENS:CORR:TYPE?'))
        time.sleep(2)
        query=str(con.query('SENS:CORR:TYPE?'))
        #print ('#####',query)
        #query='qwe'
        if ("RESPO" in query and str(port) in query):
            print(f'Calibration RO {port} succes')
        else:
            print(f'Calibration RO {port} failed')
    
    
    
    
    con.write("SENSe:SWEep:TYPE LINear")
    check_error()
    
    

In [ ]:
display(Markdown(f'### 💠 Калибровка RO'))
for port in range(1,portCount+1):
    kalibration_ro_for_port_list(inst,[j for j in range(port,port+1)])


In [ ]:
button2 = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
display(button2)

In [ ]:
%blockrun button2

In [ ]:
inst.write("SYST:PRES")

In [ ]:
def kalibration_full1_for_port_list(con,ports_list:list):
    
    con.write(f"SENS:CORR:COLL:CKIT 2")
    check_error()    
    for port in ports_list:     
        #print(f"SENS1:CORR:COLL:OPEN {port}")
        con.write(f"SENS1:CORR:COLL:OPEN {port}")
        check_error()
        #print(f"SENS1:CORR:COLL:METH:OPEN {port}")        
        con.write(f"SENS1:CORR:COLL:SHORT {port}")
        check_error()        
        con.write(f"SENS1:CORR:COLL:LOAD {port}")
        check_error()
        con.write(f'SENS1:CORR:COLL:METH:SOLT1 {port}')
        con.write("SENS:CORR:COLL:SAVE")         
        check_error()
        con.write(f"CALC:PAR1:DEF s{port}{port}") 
        #print("SENS:CORR:COLL:SAVE")
        #print(con.query(f'SENS:CORR:TYPE?'))
        query=str(con.query('SENS:CORR:TYPE?'))
        print ('#####',query)
        if ("SOLT1" in query and str(port) in query):
            print(f'Calibration SOLT1 {port} succes')
        else:
            print(f'Calibration SOLT1 {port} failed')
    
    
    
    
    con.write("SENSe:SWEep:TYPE LINear")
    check_error()
    
#    прерводим прибор в режим внешнего управления и останавливаем развёртку

In [ ]:
display(Markdown(f'### 💠 Калибровка Full1'))
for port in range(1,portCount+1):
    kalibration_full1_for_port_list(inst,[j for j in range(port,port+1)])

In [ ]:
button3 = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
display(button3)

In [ ]:
%blockrun button3

In [ ]:
inst.write("SYST:PRES")

In [ ]:
def kalibration_full2_for_port_list(con,ports_list:list):
    print(ports_list)
    con.write(f"SENS:CORR:COLL:CKIT 3")
    check_error()    
    for port in ports_list:     
        #print(f"SENS1:CORR:COLL:OPEN {port}")
        con.write(f"SENS1:CORR:COLL:OPEN {port}")
        check_error()
        #print(f"SENS1:CORR:COLL:METH:OPEN {port}")        
        con.write(f"SENS1:CORR:COLL:SHORT {port}")
        check_error()        
        con.write(f"SENS1:CORR:COLL:LOAD {port}")
        check_error()
    con.write(f"SENS1:CORR:COLL:THRU 1,2")
    con.write(f"SENS1:CORR:COLL:THRU 2,1")
    check_error()
    con.write(f'SENS1:CORR:COLL:METH:SOLT2 1,2')
    con.write("SENS:CORR:COLL:SAVE")         
    check_error()
    con.write(f"CALC:PAR1:DEF s{port}{port}") 
    #print("SENS:CORR:COLL:SAVE")
    #print(con.query(f'SENS:CORR:TYPE?'))
    query=str(con.query('SENS:CORR:TYPE?'))
    print ('#####',query)
    if ("SOLT1" in query and str(port) in query):
        print(f'Calibration SOLT1 {port} succes')
    else:
        print(f'Calibration SOLT1 {port} failed')
    
    
    
    
    con.write("SENSe:SWEep:TYPE LINear")
    check_error()
    
#    прерводим прибор в режим внешнего управления и останавливаем развёртку

In [ ]:
display(Markdown(f'### 💠 Калибровка Full2 1,2 port'))
for port in range(1,3):
   #print(f"SENS1:CORR:COLL:OPEN {port}")
    inst.write(f"SENS1:CORR:COLL:OPEN {port}")
    check_error()
    #print(f"SENS1:CORR:COLL:METH:OPEN {port}")        
    inst.write(f"SENS1:CORR:COLL:SHORT {port}")
    check_error()        
    inst.write(f"SENS1:CORR:COLL:LOAD {port}")
    check_error()
inst.write(f"SENS1:CORR:COLL:THRU 1,2")
inst.write(f"SENS1:CORR:COLL:THRU 2,1")
check_error()
inst.write(f'SENS1:CORR:COLL:METH:SOLT2 1,2')
inst.write("SENS:CORR:COLL:SAVE")         
check_error()
inst.write(f"CALC:PAR1:DEF s11") 
#print("SENS:CORR:COLL:SAVE")
#print(con.query(f'SENS:CORR:TYPE?'))
query=str(inst.query('SENS:CORR:TYPE?'))
print ('#####',query)
if ("SOLT2" in query):
    print(f'Calibration SOLT2  succes')
else:
    print(f'Calibration SOLT2  failed')
inst.write("SENSe:SWEep:TYPE LINear")
check_error()


In [ ]:
button4 = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
display(button4)

In [ ]:
%blockrun button4

In [ ]:
inst.write("SYST:PRES")

In [ ]:
display(Markdown(f'### 💠 Калибровка Full3 1,2,3 port'))
for port in range(1,4):
   #print(f"SENS1:CORR:COLL:OPEN {port}")
    inst.write(f"SENS1:CORR:COLL:OPEN {port}")
    check_error()
    #print(f"SENS1:CORR:COLL:METH:OPEN {port}")        
    inst.write(f"SENS1:CORR:COLL:SHORT {port}")
    check_error()        
    inst.write(f"SENS1:CORR:COLL:LOAD {port}")
    check_error()
inst.write(f"SENS1:CORR:COLL:THRU 1,2")
inst.write(f"SENS1:CORR:COLL:THRU 1,3")
inst.write(f"SENS1:CORR:COLL:THRU 2,1")
inst.write(f"SENS1:CORR:COLL:THRU 2,3")
inst.write(f"SENS1:CORR:COLL:THRU 3,1")
inst.write(f"SENS1:CORR:COLL:THRU 3,2")
check_error()
inst.write(f'SENS1:CORR:COLL:METH:SOLT3 1,2,3')
inst.write("SENS:CORR:COLL:SAVE")         
check_error()
inst.write(f"CALC:PAR1:DEF s11") 
#print("SENS:CORR:COLL:SAVE")
#print(con.query(f'SENS:CORR:TYPE?'))
query=str(inst.query('SENS:CORR:TYPE?'))
print ('#####',query)
if ("SOLTN" in query):
    print(f'Calibration SOLT3  succes')
else:
    print(f'Calibration SOLT3  failed')
inst.write("SENSe:SWEep:TYPE LINear")
check_error()


In [ ]:
button5 = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
display(button5)

In [ ]:
%blockrun button5

In [ ]:
inst.write("SYST:PRES")

In [ ]:
display(Markdown(f'### 💠 Калибровка Full4 1,2,3,4 port'))
for port in range(1,5):
   #print(f"SENS1:CORR:COLL:OPEN {port}")
    inst.write(f"SENS1:CORR:COLL:OPEN {port}")
    check_error()
    #print(f"SENS1:CORR:COLL:METH:OPEN {port}")        
    inst.write(f"SENS1:CORR:COLL:SHORT {port}")
    check_error()        
    inst.write(f"SENS1:CORR:COLL:LOAD {port}")
    check_error()
inst.write(f"SENS1:CORR:COLL:THRU 1,2")
inst.write(f"SENS1:CORR:COLL:THRU 1,3")
inst.write(f"SENS1:CORR:COLL:THRU 1,4")
inst.write(f"SENS1:CORR:COLL:THRU 2,1")
inst.write(f"SENS1:CORR:COLL:THRU 2,3")
inst.write(f"SENS1:CORR:COLL:THRU 2,4")
inst.write(f"SENS1:CORR:COLL:THRU 3,1")
inst.write(f"SENS1:CORR:COLL:THRU 3,2")
inst.write(f"SENS1:CORR:COLL:THRU 3,4")
inst.write(f"SENS1:CORR:COLL:THRU 4,1")
inst.write(f"SENS1:CORR:COLL:THRU 4,2")
inst.write(f"SENS1:CORR:COLL:THRU 4,3")
check_error()
inst.write(f'SENS1:CORR:COLL:METH:SOLT4 1,2,3,4')
inst.write("SENS:CORR:COLL:SAVE")         
check_error()
inst.write(f"CALC:PAR1:DEF s11") 
#print("SENS:CORR:COLL:SAVE")
#print(con.query(f'SENS:CORR:TYPE?'))
query=str(inst.query('SENS:CORR:TYPE?'))
print ('#####',query)
if ("SOLTN" in query):
    print(f'Calibration SOLT4  succes')
else:
    print(f'Calibration SOLT4  failed')
inst.write("SENSe:SWEep:TYPE LINear")
check_error()

In [ ]:
inst.write("SENSe:CORRection:COLLect:CKIT:RESet")